# Molecular Mechanics Optimization of Molecular Systems

## Table of Content <a name="TOC"></a>

1. [General setups](#setups)


2. [Regular optimization example](#2.) 


3. [Capping example](#3.)   


### A. Learning objectives

- to optimize the structure of a random molecule using classical MM in Libra
- to construct a new molecule by adding new atoms to the existins molecule and linking atoms
- to optimize the guessed structure of the new molecule


### B. Use cases

- molecular mechanics calculations (energy & forces)
- structure optimization
- simulated annealing
- building molecular structure


### C. Functions

- `libra_py`  
  - `nve_md`
    - [`optimize_syst`](#optimize_syst-1) | [`optimize_syst`](#optimize_syst-2)
    - [`syst2xyz`](#syst2xyz-1)
          

### D. Classes and class members

- `liblibra::libchemobjects`
  - `libchemsys`
    - `System`
      - [`CREATE_ATOM`](#CREATE_ATOM-1)        
      - [`LINK_ATOMS`](#LINK_ATOMS-1)
      

## 1. General setups
<a name="setups"></a>[Back to TOC](#TOC)

Let's import all the needed libraries, including the `py3Dmol` library for visualizing the structures we generate.

In [1]:
import sys
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *

from libra_py import LoadPT
from libra_py import LoadMolecule
from libra_py import nve_md
from libra_py import units

import py3Dmol   # molecular visualization


The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::v

## 2. Regular optimization example
<a name="2."></a>[Back to TOC](#TOC)

Let's load the system from the file and save it as the first snapshot. 

Once a chemical system object is created it can always be saved as an xyz string (and written to the corresponding file) using the `syst2xyz` function.
<a name="syst2xyz-1"></a>

In [2]:
U = Universe()
LoadPT.Load_PT(U, "elements.dat", 0)

syst = System()    
LoadMolecule.Load_Molecule(U, syst, "test1a.pdb", "pdb_1")    

xyz = nve_md.syst2xyz(syst)
print(xyz)

24
 11 

C  2.409 0.661 0.000
H  3.084 1.614 1.652
H  3.084 1.614 -1.652
H  0.387 0.661 0.000
C  3.379 -2.082 0.000
H  2.706 -3.035 -1.652
H  5.401 -2.082 -0.000
C  2.409 -3.454 2.375
H  3.086 -2.504 4.027
H  0.387 -3.451 2.377
F  3.256 -5.860 2.373



Let's run the optimization recipe on this molecule and save the final snapshot.

Under the hood, what the `optimize_syst` function is doing - is running the short MD trajectories followed by the cooling (resetting momenta to zero, hence taking away the excess kinetic energy). During the MD part, the system naturally converts its potential energy to the kinetic one, thus moving down on the PES. The cooling step takes away the produced heat (kinetic energy), hence making the process unidirectional in general.

Here, the calculations are run with the default parameters as signified by the empty dictionary used as the second parameter.
<a name="optimize_syst-1"></a>

In [3]:
nve_md.optimize_syst(syst, {})

xyz2 = nve_md.syst2xyz(syst)
print(xyz2)

 11 

C  2.423 0.670 -0.026
H  3.121 1.714 1.657
H  3.144 1.620 -1.752
H  0.324 0.727 -0.056
C  3.350 -2.059 0.033
H  2.667 -3.057 -1.684
H  5.450 -2.075 0.019
C  2.399 -3.438 2.381
H  3.095 -2.480 4.115
H  0.300 -3.464 2.406
F  3.274 -5.900 2.368



We could have supplied non-default parameters, according to the expectations:

In [4]:
help(nve_md.optimize_syst)

Help on function optimize_syst in module libra_py.nve_md:

optimize_syst(syst, params)
    A function to optimize the geometry of the system 
    
    Args:
        syst ( System object ): represents the chemical object
        params ( dictionary ): control parameters:
    
            * **params["anneal_schedule"]** ( list ): the annealing schedule. Each element of the list
                consist of 3 elements: dt, ncycles, nsteps, where:
                dt - the timestep for integration [ units: a.u. ]
                ncycles - the number of cycles of annealing with dt 
                Each annealing cycle consists of ```nsteps```  steps of NVE MD steps followed by the cooling
                Cooling just resets all the momenta and angular momenta to zero
                Example:
                    params["anneal_schedule"] = [ [1.0, 100, 10], [20.0, 100, 100] ] means:
                    First do 100 cycles of annealing: 10 MD steps with dt = 1 a.u. each followed by cooling
     

So an example would be:
<a name="optimize_syst-2"></a>

In [5]:
ann_schedule = [  [20.0, 100, 10],
                  [40.0, 100, 10],
                  [40.0, 100, 100]
]

nve_md.optimize_syst(syst, { "anneal_schedule":ann_schedule })

xyz2 = nve_md.syst2xyz(syst)
print(xyz2)

 11 

C  2.423 0.671 -0.025
H  3.122 1.714 1.657
H  3.145 1.620 -1.752
H  0.325 0.728 -0.055
C  3.350 -2.059 0.033
H  2.666 -3.057 -1.684
H  5.450 -2.076 0.019
C  2.399 -3.438 2.381
H  3.094 -2.480 4.115
H  0.300 -3.465 2.406
F  3.275 -5.900 2.368



Using the initial and final snapshots, lets animate the transition between the two structures

In [6]:
do_show = 1
if do_show:
    xyzall = xyz+xyz2

    view = py3Dmol.view(width=800,height=400)  # linked=False,viewergrid=(3,2)\n
    view.setBackgroundColor('0xeeeeee')                                     
    view.zoomTo()                                                           
    view.addModelsAsFrames(xyzall, 'xyz')
    view.setStyle({'sphere':{'colorscheme':'Jmol', }})                        
    view.animate({'reps':0, 'loop':'backandforward', 'step':1})
    view.show() 

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## 3. Capping example
<a name="3."></a>[Back to TOC](#TOC)

Let's do all the same, but for the system with the fluorine "F" atom removed

In [7]:
syst = System()    
LoadMolecule.Load_Molecule(U, syst, "test1b.pdb", "pdb_1")    
xyz = nve_md.syst2xyz(syst)
print(xyz)

22
 10 

C  2.409 0.661 0.000
H  3.084 1.614 1.652
H  3.084 1.614 -1.652
H  0.387 0.661 0.000
C  3.379 -2.082 0.000
H  2.706 -3.035 -1.652
H  5.401 -2.082 -0.000
C  2.409 -3.454 2.375
H  3.086 -2.504 4.027
H  0.387 -3.451 2.377



Now, lets try to attach a new atom and optimize the resulting structure

Beware: if one optimizes the original structure and then adds new atoms and optimize again - this doesn't work correctly.
So, for now, we need to first guess the location of the capping atom first, then add it to the system and only then - optimize.
<a name="CREATE_ATOM-1"></a><a name="LINK_ATOMS-1"></a>

In [8]:
rnd = Random()
d = 2.0 * units.Angst

x, y, z = 0.205* units.Angst,  -1.826* units.Angst,   1.258* units.Angst - d
syst.CREATE_ATOM( Atom(U, {"Atom_element":"F", "Atom_cm_x":x, "Atom_cm_y":y, "Atom_cm_z":z }  )  )
syst.LINK_ATOMS(8, 11)

xyz0 = nve_md.syst2xyz(syst)
print(xyz0)

 11 

C  2.409 0.661 0.000
H  3.084 1.614 1.652
H  3.084 1.614 -1.652
H  0.387 0.661 0.000
C  3.379 -2.082 0.000
H  2.706 -3.035 -1.652
H  5.401 -2.082 -0.000
C  2.409 -3.454 2.375
H  3.086 -2.504 4.027
H  0.387 -3.451 2.377
F  0.387 -3.451 -1.402



Let's finally optimize the guessed structure.

The option `fixed_fragment_translation` is used to not move all the atoms other than the newly added one

In [9]:
nve_md.optimize_syst(syst, {"anneal_schedule":[ [1.0, 10, 100], [10.0, 10, 100] ], 
                            "fixed_fragment_translation":range(0,10) })
xyz3 = nve_md.syst2xyz(syst)
print( xyz3)

 11 

C  2.409 0.661 0.000
H  3.084 1.614 1.652
H  3.084 1.614 -1.652
H  0.387 0.661 0.000
C  3.379 -2.082 0.000
H  2.706 -3.035 -1.652
H  5.401 -2.082 0.000
C  2.409 -3.454 2.375
H  3.086 -2.504 4.027
H  0.387 -3.451 2.377
F  3.345 -6.178 2.385



Visualize the result

In [10]:
do_show = 1

if do_show:    
    xyzall2 = xyz0+xyz3

    view = py3Dmol.view(width=800,height=400, linked=False,viewergrid=(1,3)) 
    view.setBackgroundColor('0xeeeeee')                                     
    view.zoomTo()                                                           
    view.addModelsAsFrames(xyzall2, 'xyz', viewer=(0,0))
    view.addModel(xyz0,'xyz',{'vibrate': {'frames':10,'amplitude':1.0}}, viewer=(0,1))
    view.addModel(xyz3,'xyz',{'vibrate': {'frames':10,'amplitude':1.0}}, viewer=(0,2))
    view.setStyle({'sphere':{'colorscheme':'Jmol', }})                        
    view.animate({'reps':0, 'loop':'backandforth', 'step':1})
    view.show() 

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol